# OpenStreetMap Data Case Study

## Table of Contents:

1. Project Introduction
2. Project Overview
3. Map Area: Edinburgh, Scotland
4. Problems Encountered in the Map
5. Overview of the Data
6. Other Ideas About the Dataset

## 1. Project introduction

The purpose of this project is to download a geographic section of the openstreetmap data, clean the data, then export it to a database so a series of queries can be asked to reveal interesting information about the map.

## 2. Project Overview

The order of how this project was handled is as follows:

1. Download map from openstreetmap as an osm file
2. Parse the osm file using ElementTree module
3. Look at the structure of the data and discover errors
4. Fix errors (if possible)
5. Export data by type of tag (node, node_tags, way, way_tags, way_nodes) as a csv file
6. Store the csv files in a database
7. Create queries to reveal interesting information about the data

## 3. Map Area: Edinburgh, Scotland

* OpenStreetMap address: https://www.openstreetmap.org/node/17898859#map=10/55.9504/-3.1915
* Donwload address: https://mapzen.com/data/metro-extracts/metro/edinburgh_scotland/85684587/City%20of%20Edinburgh/

I originally wanted to use Seoul or Bangkok, cities which are both more familiar to me, but both maps contained a complicated mess with both English and local language used. So for this project, I decided to look into the city I enjoyed visiting very much, Edinburgh.

## 4. Problems Encountered in the Map

### 1. Street names

### 1a. Abbreviated street names

Some street names were abbreviated, or spelled wrong. I first ran a function that looked at all street types (most street types were mentioned at the end of the address), and looked at each one and decided whether they were faulty or not. At times I had to search on Google, because they were local street types that wasn't used elsewhere (e.g. Brae, Wynd).

#### 1b. Directions within the street name

While running the function mentioned above to look for faulty street names, I came across numerous addresses with directions at the end of the address: 'North', 'South', West', and 'East'. I noticed that they were just referring to part of a longer street, and that the actual addresses were just the street name without the direction (e.g. the area may be referred as 'Stenhouse Street East', but is actually part of the 'Stenhouse Street'. The street type names I wanted were 'hidden' before these directions, so I decided to get rid of these words.


#### 1c. Other street name problems
While the Edinburgh map's data was surprisingly well documented, there were obvious names that were probably not 'street' types, but probably a 'property' type. The below includes some of these examples.

In [1]:
property_names = ['buildings', 'cottages', 'estate', 'garden(s)',
                 'holdings', 'mall', 'manor', 'market', 'mills',
                 'park', 'quadrant', 'port', 'steading(s)',
                 'village', 'villas', 'yard']

The problem with these errors was that there were simply too many of them scattered throughout the map that it wasn't possible to make the changes manually one by one, as I would also have to manually look for the actual address. However, I also felt that for a quick exploratory analysis, it was okay to leave them as they are right now.

Problems a and b were corrected using the following code:

This is a sample of how the street names changed:

* `'Duddingston Gardens North' -> 'Duddingston Gardens'`
* `'Railway Path South' -> 'Raily Path'`
* `'St James place' -> 'St James Place'`
* `'Mayburn Ave' -> 'Mayburn Avenue'`
* `'Logie Green Rd' -> 'Logie Green Road'`

### 2. Postal codes
Edinburgh's postal codes start with 'EH', followed by a single or two digit number, followed by a single space, followed by a number, and two capitalized alphabets.

In the data, postcodes were located under two different attribute names: 

`postal_code` and `addr:postcode`.

I used the following regex expression to find postal codes that didn't match the format.

`find_postal = re.compile(r'(EH\d{1,2} \d{1}[A-Z][A-Z])')`

Surprisingly, the postal codes showed no serious errors. There were a group of 'KY' postal codes, which is a postcode for an upper region of Edinburgh. My guess is that since these other cities were very close to Edinburgh, they were mistakenly included as part of Edinburgh's map data. I decided just to leave them for this analysis.

### 3. City
As in the case of postal codes, because there were some locations that are not part of Edinburgh included in the data, there are several towns and cities included in this data.

I looked through the attribute `addr:city` and searched for cities that are not `'Edinburgh'`.

Likewise in postal code, I decided not to remove the other cities, but I did found some faulty names and spelling errors, so made a mapping method similar to updating street name to change these errors.

### 4. Phone numbers
Phone numbers showed one of the most diverse formats. First of all, phone numbers were distributed in attributes `phone` and `contact:phone`, as in the case of postal codes.

In the end, I decided to follow the UK phone format standard as detailed in 'https://www.ukphoneinfo.com'.

I used the following regex expression to look for all phone numbers that are not standard Edinburgh phone number format.

`phone_check = re.compile(r'(0131) \d\d\d \d\d\d\d')`

This is a sample of how the phone numbers changed:

* `+44 131 447 9027 -> 0131 447 9027`
* `+44 (0)131 656 0390 -> 0131 656 0390`
* `01316542777 -> 0131 654 2777`
* `+44 788 983 2780 -> 07889 832780`
* `+44 131 5525522 -> 0131 552 5522`

## 5. Overview of the Data

### File sizes

#### Original osm file

* `edinburgh_scotland.osm 469.9 MB`

#### Individual csv files
* `nodes.csv 154.9 MB`
* `node_tags.csv 26.8 MB`
* `ways.csv 22.2 MB `
* `way_tags.csv 32.8 MB`
* `way_nodes.csv 70.1 MB`

#### Database file
* `edinburgh.db 352.2 MB`

## Number of entries in each file

```sql
SELECT COUNT(*) FROM nodes;
```
<strong>```number of nodes: ```</strong>```1948773```

```sql
SELECT COUNT(*) FROM node_tags
```
<strong>```number of node_tags: ```</strong>```814471```

```sql
SELECT COUNT(*) FROM ways
```
<strong>```number of ways: ```<strong>```380002```

```sql
SELECT COUNT(*) FROM way_tags
```
<strong>```number of way_tags: ```</strong>```909648```

```sql
SELECT COUNT(*) FROM way_nodes
```
<strong>```number of way_nodes: ```</strong>```3071078```

### Number of unique users: 

```sql
SELECT COUNT(DISTINCT(e.uid)) 
FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) e;
```

<strong>```number of unique users:```</strong> ```1087```

### Top 10 contributing users

```sql
SELECT e.user, COUNT(*) as num
FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e GROUP BY e.user ORDER BY num DESC;
```


<strong>```top 10 contributing users:```</strong>
```
sophiemccallum|563943
drnoble|318883
eric_|224366
rob_michel|138985
Hobgoblin|130447
Central America|95904
michaelbrown|92752
eisa|83557
GinaroZ|81735
leilaz|41564
```

### Types available for the dataset

```sql
SELECT e.type, COUNT(*) as num
FROM (SELECT type FROM node_tags UNION ALL SELECT type FROM way_tags) e 
GROUP BY e.type ORDER BY num DESC LIMIT 10;
```

<strong>```output:```</strong>
```
regular|1283261
addr|390278
naptan|24402
building|11817
maxspeed|3262
roof|2641
source|2454
seamark|993
recycling|614
disused|537
```

### 'Regular' type keys

For those addresses that didn't have a specific address type, their key values were assigned as 'regular'. There are 1.2 million 'regular' types here, so it only makes sense to dig into them a bit more.

```sql
SELECT e.key, COUNT(*) as num FROM (SELECT type, key FROM node_tags UNION ALL SELECT type, key FROM way_tags) e WHERE e.type = 'regular' GROUP BY e.key ORDER BY num DESC LIMIT 10;
```

<strong>```output:```</strong>
```
source|447388
building|171525
natural|90267
highway|74693
denotation|69674
barrier|67002
access|51557
leisure|51111
name|45209
maxspeed|21603
```

### sources of the data

```sql
SELECT e.value, COUNT(*) as num FROM (SELECT type, value FROM node_tags UNION ALL SELECT type, value FROM way_tags) e WHERE e.type = 'regular' GROUP BY e.value ORDER BY num DESC LIMIT 10;
```

<strong>```output:```</strong>
```
Bing|154148
NLS_OS_Edinburgh_map_1940s;Bing|128836
survey|93244
tree|82582
yes|78432
urban|69659
residential|55376
wall|54994
private|50341
garden|47935
```

Entries with this type seem to be indicate the original source of the data. Bing, NLS (National Library of Scotland), and survey seems to be the most referenced source.

### 'abandoned' places

There are 36 'abandoned' type locations in Edinburgh. I was curious, so I dug in a bit more.

```sql
SELECT e.key, COUNT(*) as num
FROM (SELECT key, type FROM node_tags UNION ALL SELECT key, type FROM way_tags) e 
WHERE e.type = 'abandoned' GROUP BY e.key ORDER BY num DESC;
```

<strong>```output:```</strong>
```
railway|22
highway|8
amenity|2
building|2
leisure|1
man_made|1
```

Ah. There are numerous unused railways and highways in the area. It makes sense then to mark these as 'abandoned' rather than to ignore them because they take a huge space just to be ignored from the map.

### Keys available for data

```sql
SELECT e.key, COUNT(*) as num
FROM (SELECT key FROM node_tags UNION ALL SELECT key FROM way_tags) e 
GROUP BY e.key ORDER BY num DESC LIMIT 10;
```

<strong>```output:```</strong>
```
source|447472
building|171608
street|101096
housenumber|100078
city|92180
natural|90267
country|77209
highway|74862
denotation|69674
barrier|67015
```

There are some interesting key values here. Let's look at some in detail.

### History vs Historic

We see history, we see historic. let's see both to see if they are any different.

```sql
SELECT e.value, e.key, COUNT(*) as num
FROM (SELECT value, key FROM node_tags UNION ALL SELECT value, key FROM way_tags) e
WHERE (e.key = 'history' OR e.key = 'historic') GROUP BY e.value ORDER BY num DESC;
```

<strong>```output:```</strong>
```
memorial|historic|136
ruins|historic|112
monument|historic|105
Retrieved from v1|history|53
archaeological_site|historic|43
Retrieved from v2|history|35
Retrieved from v3|history|28
castle|historic|27
mine|historic|27
tomb|historic|21
```

It seems that these two are distinctively different, where historic is the actual 'historic_sites', while 'history' refers to a possible source of some kind. Changing the name would be much easier to follow.

### 'man_made' locations

Another thing that is interesting is 'man_made' keys. Possibly everything in this dataset except for natural locations should be man_made. Why are they separately written?

```sql
SELECT e.value, COUNT(*) as num
FROM (SELECT value, key FROM node_tags UNION ALL SELECT value, key FROM way_tags) e
WHERE e.key = 'man_made' GROUP BY e.value ORDER BY num DESC LIMIT 10;
```

<strong>```output:```</strong>
```
embankment|238
pier|76
tower|65
mast|48
petroleum_well|43
surveillance|40
wastewater_plant|24
cutline|22
pipeline|19
tank|18

```

The results are not something special. Clearly, these entries can be submitted under different keys which are more coherent.



I suspect that a particular user (or users) might be the culprit and looked at who made these entries.

```sql
SELECT e.user, COUNT(*) as num
FROM (SELECT user, id FROM nodes UNION ALL SELECT user, id FROM ways) e JOIN (SELECT id, key FROM node_tags UNION ALL SELECT id, key FROM way_tags) f ON e.id = f.id WHERE f.key = 'man_made' GROUP BY e.user ORDER BY num DESC LIMIT 10;
```

<strong>```output:```</strong>
```
drnoble|309
GinaroZ|61
kaerast|38
Central America|31
Hobgoblin|28
eric_|25
sophiemccallum|19
phut|17
digby|15
MJCosm|14
```

But it seems that pretty much anyone makes this mistake. In fact, some of the top contributing users also make this mistake.

### Tourist sites

Edinburgh is famous for its tourist locations, so I'm interested in these sites. Since historic locations are a big part of Edinburgh's history, I'm also curious if there are tourist locations that are listed as such, and thus have a potential of overlapping with 'historic' key addresses.

```sql
SELECT e.value, COUNT(*) as num
FROM (SELECT value, key FROM node_tags UNION ALL SELECT value, key FROM way_tags) e
WHERE e.key = 'tourism' GROUP BY e.value ORDER BY num DESC LIMIT 10;
```

<strong>```output:```</strong>
```
guest_house|235
hotel|232
attraction|204
information|199
artwork|109
picnic_site|85
viewpoint|49
museum|35
hostel|29
gallery|24
```

### Religions in Edinburgh

```sql
SELECT e.value, COUNT(*) as num
FROM (SELECT value, key FROM node_tags UNION ALL SELECT value, key FROM way_tags) e
WHERE e.key = 'religion' GROUP BY e.value ORDER BY num DESC;
```

<strong>```output:```</strong>
```
christian|384
muslim|8
multifaith|5
jewish|2
bahai|1
buddhist|1
hindu|1
sikh|1
```

not a surprising result, as this is Scotland.

### Common ammenities in Edinburgh

```
SELECT e.value, COUNT(*) as num
FROM (SELECT value, key FROM node_tags UNION ALL SELECT value, key FROM way_tags) e
WHERE e.key = 'amenity' GROUP BY e.value ORDER BY num DESC LIMIT 10;
```

<strong>```output:```</strong>
```
parking|3836
bench|1208
post_box|1090
bicycle_parking|1071
restaurant|728
fast_food|661
cafe|633
pub|504
telephone|477
place_of_worship|383
```

We see restaurants and fast_food chains as part of amenities, but interestingly, there is also a separate key for 'cuisine', which most likely is a resturant too.

```
SELECT e.value, COUNT(*) as num
FROM (SELECT value, key FROM node_tags UNION ALL SELECT value, key FROM way_tags) e
WHERE e.key = 'cuisine' GROUP BY e.value ORDER BY num DESC LIMIT 10;
```

<strong>```output:```</strong>
```
chinese|180
indian|126
italian|96
fish_and_chips|93
coffee_shop|69
sandwich|59
pizza|50
burger|31
thai|29
mexican|23
```

Perhaps those amenities listed as restaurants are more common local cuisines that don't need a separate value like 'chinese' or 'sandwich'. We see a lot of foreign cuisines here, while there are only 18 regional restaurants. Still, interesting find.

## 6. Other Ideas About the Dataset

### creating new attributes
'source' is currently part of the 'type' tag for some addresses. While this may be helpful, it's not really contributing anything to do the list. I suggest there should be a separate attribute indicating the source. This will help organize the data more, and furthermore, it would ensure verification of the data.

The formats for submitting specific sources would have to be set too, but this can be complex, as we cannot simply predict what the sources may be. Even from this data alone, we can see that some are online, some are from surveys, and some are from individual accounts.

### abbreviation/typo/capitalization checks
Some commonly found mistakes could be mapped before added to the dataset. It would also be helpful if a script can be created catching these unusualities and report to the user to ensure that the unusuality is not a mistake. 

Restricting certain names may work too, but there may be rare cases where those restrictions are actual names of the street address. 

### wrong entries
There are some value/key/type entries that should be entered for other variables. Going back to the 'sources' type again, values such as 'garden', 'wall', 'residential' are clearly 'types' of a certain construction, rather than a value for sources. 

When looking at they keys, I saw some keys such as 'opening_hours' or 'collection_times' that are complimentary information regarding the said place, rather than a key.

There were also some really absurd values. One of the 'key' groups was 'horse', and since the values are plain 'yes' and 'no', I'm only to assume that these are part of a survey question. Still, very strange values for a street data.

These should be monitored regularly to maintain consistency within the data.

We may build some simple rules to allow only certain keys and values to be entered. The biggest problem, of course, would be to decide beforehand what values can be accepted, and this 'acceptable' values alone can be too much in number to manually handle.

### need for rigid formatting
While most values can be freely submitted, especially if they are text values, many can be restricted in terms of formatting. For example, valid phone number within a geographic region will probably have limited number of formats used. Same can be said for other unique codes, such as postal codes or house numbers. 

However, this may be hindered by the fact that some businesses use mobile phone number as main contact point, and mobile numbers are not region specific. And as for any rigid formatting, if there are any changes in the formatting, these rules would need to be added manually, which is not only tiresome, but if the rules were applied wrong, then the resulting data may be rigidly formatted trash data.